In [ ]:
import pims
import dask.array as da
from dask import delayed, compute
import numpy as np

In [ ]:
filename = '../tests/data/testData3series.lif'

In [ ]:
def get_lif_metadata(filename):
    with pims.Bioformats(filename, read_mode="jpype") as lifSeq:
        xscale = lifSeq.metadata.PixelsPhysicalSizeX(0)
        tscale = lifSeq.metadata.PlaneDeltaT(0, 1) * 1000.0
        lifXCoords = np.arange(0.0, lifSeq.shape[1] * xscale, xscale)
        lifYCoords = np.arange(0.0, lifSeq.shape[2] * xscale, xscale)
        lifTCoords = np.arange(0.0, lifSeq.shape[0] * tscale, tscale)
        return (lifXCoords, lifYCoords, lifTCoords)

In [ ]:
get_lif_metadata(filename)

In [ ]:
lifSeq = pims.Bioformats(filename, read_mode="jpype")

In [ ]:
lifSeq.get_frame(1)

In [ ]:
def get_frame(filename, frame, chunk):
    frames = range(frame, frame+chunk)
    with pims.Bioformats(filename, read_mode="jpype") as lifSeq:
        arr = [lifSeq.get_frame(frame) for frame in frames]
        d_arr = da.from_array(arr, chunks=(chunk, -1, -1))
        return d_arr

In [ ]:
arr = get_frame(filename, 3, 2)
arr

In [ ]:
chunk_size = 2
arr = [get_frame(filename, frame, chunk_size) for frame in range(1,10,chunk_size)]
out = da.stack(arr)

In [1]:
from dask.array import map_blocks

In [2]:
from nd2 import ND2File
f = ND2File('../data/A1_s3001.nd2')

In [8]:
filename = '../data/A1_s3001.nd2'
def to_dask(filename, chunk_size = 5):
    f = ND2File(filename)
    shape = (int(f._coord_shape[0]/chunk_size), )
    chunks = [(chunk_size,) * x for x in shape]
    chunks += [(x,) for x in f._frame_shape]
    dask_arr = map_blocks(
        f._dask_block,
        copy=True,
        chunks=chunks,
        dtype=f.dtype,
    )
    return dask_arr

In [9]:
to_dask(filename)

dask.array<_dask_block, shape=(5000, 512, 512), dtype=uint16, chunksize=(5, 512, 512), chunktype=numpy.ndarray>